In [7]:
import pyodbc
import reddit_engine as rpe 
import importlib
import time

# Force le rechargement du module .py si vous faites des changements
importlib.reload(rpe)

<module 'reddit_engine' from 'c:\\Users\\stgadmin\\Desktop\\MarketReviews_VS_TRUTH\\reddit_engine.py'>

In [8]:
print("--- Lancement de l'Extraction (Étape 6a) ---")
print("Objectif : Lire Dim_Product depuis le DWH.")

conn_dwh = None
cursor_dwh = None
product_list_to_search = [] # La variable qui stockera notre liste de travail

try:
    conn_dwh = rpe.get_dwh_connection()
    cursor_dwh = conn_dwh.cursor()
    print("   -> ✅ Connecté au DWH.")

    # Appelle la fonction de votre .py
    product_list_to_search = rpe.get_products_to_search(cursor_dwh)
    
    if product_list_to_search:
        print("\n--- ✅ SUCCÈS : Liste des produits à rechercher extraite ---")
        print(f"{len(product_list_to_search)} produits au total.")
        print("Voici les 5 premiers produits :")
        for product in product_list_to_search[:5]:
            print(f"   - Key: {product.ProductKey}, Nom: {product.ProductName}")
    else:
        print("\n--- ⚠️ AVERTISSEMENT : Aucun produit trouvé dans Dim_Product. ---")

except Exception as e:
    print(f"\n--- ❌ ERREUR : {e} ---")
finally:
    if cursor_dwh: cursor_dwh.close()
    if conn_dwh: conn_dwh.close()
    print("\nConnexion DWH fermée. Script d'extraction terminé.")

--- Lancement de l'Extraction (Étape 6a) ---
Objectif : Lire Dim_Product depuis le DWH.
   -> ✅ Connecté au DWH.
   -> (E) Extraction des produits depuis Dim_Product...
   -> 1299 produits trouvés à rechercher sur Reddit.

--- ✅ SUCCÈS : Liste des produits à rechercher extraite ---
1299 produits au total.
Voici les 5 premiers produits :
   - Key: 2, Nom: PHILIPS THE ONE AMBILIGHT QLED 4K 43 POUCES 43PUS9050/12 (2025)
   - Key: 6, Nom: SAMSUNG OLED 4K 65 POUCES QE65S95FATXXN (2025)
   - Key: 7, Nom: JVC LT-43FV140
   - Key: 8, Nom: SAMSUNG QLED 4K 50 POUCES QE50Q7F3AUXXN (2025)
   - Key: 10, Nom: SAMSUNG OLED 4K 77 POUCES QE77S95FATXXN (2025)

Connexion DWH fermée. Script d'extraction terminé.


In [6]:
# --- Cellule 3 : Étape 6b/6c (Boucle complète) ---
print("\n--- Lancement du Scraping/Chargement (Étape 6b & 6c) ---")

if 'product_list_to_search' not in locals() or not product_list_to_search:
    print("--- ❌ ERREUR : La variable 'product_list_to_search' est vide. ---")
    print("   Veuillez (ré)exécuter la cellule précédente pour extraire la liste des produits.")
else:
    
    total_products = len(product_list_to_search)
    print(f"\n--- Lancement de la boucle de scraping pour les {total_products} produits RESTANTS ---")

    for i, product in enumerate(product_list_to_search):
        product_key = product.ProductKey
        product_name = product.ProductName
        brand = product.Brand
        category = product.Category
        
        conn_staging = None
        cursor_staging = None
        
        print(f"\n-------------------------------------------------")
        print(f"--- Traitement Produit {i+1}/{total_products} (Key: {product_key}) ---")
        
        try:
            # 1. Transformer (T) - Générer le mot-clé
            search_keyword = rpe.generate_search_keyword(product_name, brand)
            print(f"   -> Mot-clé généré : '{search_keyword}'")
            
            if not search_keyword:
                print("   -> ⚠️ Mot-clé vide, impossible de chercher. On ignore.")
                continue

            # 2. Transformer (T) - Trouver le bon subreddit
            target_subreddit = rpe.get_subreddit_for_category(category)
            
            # 3. Extraire (E) - Scraper avec les bons paramètres
            # La pause de 7 secondes est MAINTENANT A L'INTERIEUR de cette fonction
            raw_posts = rpe.search_reddit_api(search_keyword, subreddit=target_subreddit)
            
            if raw_posts:
                # 4. Charger (L)
                conn_staging = rpe.get_staging_connection()
                cursor_staging = conn_staging.cursor()
                rpe.save_raw_posts_to_staging(cursor_staging, raw_posts, product_key, product_name, search_keyword)
                conn_staging.commit()
                print(f"   -> ✅ Données pour '{search_keyword}' sauvegardées.")
            else:
                print(f"   -> ⚠️ Aucun post trouvé pour '{search_keyword}' dans r/{target_subreddit}.")

            # --- CORRECTION ---
            # La pause a été déplacée dans la fonction 'search_reddit_api'
            # time.sleep(7) # <-- Plus besoin de cette ligne ici

        except Exception as e:
            print(f"   -> ❌ ERREUR lors du traitement du SKU {product_name} : {e}")
            if conn_staging: conn_staging.rollback()
        finally:
            if cursor_staging: cursor_staging.close()
            if conn_staging: conn_staging.close()

    print("\n--- ✅ Boucle de scraping Reddit terminée. ---")


--- Lancement du Scraping/Chargement (Étape 6b & 6c) ---

--- Lancement de la boucle de scraping pour les 1302 produits RESTANTS ---

-------------------------------------------------
--- Traitement Produit 1/1302 (Key: 1) ---
   -> Mot-clé généré : 'LG C5 55 OLED55C56LB'
   -> (E) Appel API Reddit pour : 'LG C5 55 OLED55C56LB' dans r/Televisions
   -> 25 posts bruts trouvés.
   -> Pause de 7 secondes (respect des limites API)...
   -> (L) Chargement de 25 posts dans Staging_Reddit_Posts...
   -> 25 nouveaux posts insérés.
   -> 0 posts ignorés (doublons).
   -> ✅ Données pour 'LG C5 55 OLED55C56LB' sauvegardées.

-------------------------------------------------
--- Traitement Produit 2/1302 (Key: 2) ---
   -> Mot-clé généré : 'PHILIPS 43 43PUS9050/12 (2025)'
   -> (E) Appel API Reddit pour : 'PHILIPS 43 43PUS9050/12 (2025)' dans r/Televisions
   -> 0 posts bruts trouvés.
   -> Pause de 7 secondes (respect des limites API)...
   -> ⚠️ Aucun post trouvé pour 'PHILIPS 43 43PUS9050/12 (

KeyboardInterrupt: 